Adjust the rain augmentation method using albumentations


In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 4080


In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import yaml
import glob
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A

sns.set_theme(style="darkgrid", font_scale=1.5, font="Arial", rc={"axes.unicode_minus": False})

import torch
from ultralytics import YOLO

from PIL import Image
import shutil
import cv2

# Set seed for reproducibility
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [3]:
# Use different YOLOv11 models for all versions
model_versions = [
    "yolo11n.pt",  # YOLOv11 nano
    "yolo11s.pt",  # YOLOv11 small
    "yolo11m.pt",  # YOLOv11 medium
    "yolo11l.pt",  # YOLOv11 large
    "yolo11x.pt"   # YOLOv11 extra-large
]

In [4]:
# Training configuration
config = {
    "path": "D:/fyp/Desktop/Drone/drone_dataset",
    "train": "D:/fyp/Desktop/Drone/drone_dataset/augmented",
    "val": "D:/fyp/Desktop/Drone/drone_dataset/valid_augmented",
    "nc": 1,  # number of classes
    "names": ["drone"],
}

with open("data.yaml", "w") as file:
    yaml.dump(config, file, default_flow_style=False)

In [5]:
import wandb
wandb.disabled = True

In [6]:
def add_rain_albumentations(image, intensity=1):
    rain_params = {
        1: {"drop_length": 5, "drop_width": 1, "blur_value": 5, "rain_type": 'drizzle'},
        2: {"drop_length": 25, "drop_width": 1, "blur_value": 5, "rain_type": 'drizzle'},
        3: {"drop_length": 20, "drop_width": 1, "blur_value": 5, "rain_type": 'heavy'},
        4: {"drop_length": 30, "drop_width": 2, "blur_value": 5, "rain_type": 'heavy'},
        5: {"drop_length": 35, "drop_width": 2, "blur_value": 5, "rain_type": 'torrential'},
    }

    p = 1.0  # always apply
    transform = A.Compose([
        A.RandomRain(
            p=p,
            brightness_coefficient=0.7,
            drop_length=rain_params[intensity]["drop_length"],
            drop_width=rain_params[intensity]["drop_width"],
            blur_value=rain_params[intensity]["blur_value"],
            rain_type=rain_params[intensity]["rain_type"]
            #rain_type="drizzle" if intensity < 3 else "heavy"
        )
    ])
    return transform(image=image)["image"]

In [7]:
print("numpy verson:", np.__version__)

numpy verson: 1.26.4


In [8]:
# === Augmentation and Label Copying ===
input_image_folder = "D:/fyp/Desktop/Drone/drone_dataset/train/images"
input_label_folder = "D:/fyp/Desktop/Drone/drone_dataset/train/labels"
output_image_folder = "D:/fyp/Desktop/Drone/drone_dataset/augmented/images"
output_label_folder = "D:/fyp/Desktop/Drone/drone_dataset/augmented/labels"

os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_label_folder, exist_ok=True)

for filename in os.listdir(input_image_folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        name, ext = os.path.splitext(filename)
        image_path = os.path.join(input_image_folder, filename)
        label_path = os.path.join(input_label_folder, name + ".txt")

        image = cv2.imread(image_path)

        # Apply 5 levels of rain intensity
        for i in range(1, 6):
            augmented_image = add_rain_albumentations(image, intensity=i)

            new_image_name = f"{name}r{i}{ext}"
            new_label_name = f"{name}r{i}.txt"

            cv2.imwrite(os.path.join(output_image_folder, new_image_name), augmented_image)

            if os.path.exists(label_path):
                shutil.copy(label_path, os.path.join(output_label_folder, new_label_name))

In [9]:
# Validation datasets augmentation

# === Paths for validation set ===
input_val_image_folder = "D:/fyp/Desktop/Drone/drone_dataset/valid/images"
input_val_label_folder = "D:/fyp/Desktop/Drone/drone_dataset/valid/labels"

output_val_image_folder = "D:/fyp/Desktop/Drone/drone_dataset/valid_augmented/images"
output_val_label_folder = "D:/fyp/Desktop/Drone/drone_dataset/valid_augmented/labels"

os.makedirs(output_val_image_folder, exist_ok=True)
os.makedirs(output_val_label_folder, exist_ok=True)

# === Loop through validation images and apply rain
for filename in os.listdir(input_val_image_folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        name, ext = os.path.splitext(filename)
        image_path = os.path.join(input_val_image_folder, filename)
        label_path = os.path.join(input_val_label_folder, name + ".txt")

        image = cv2.imread(image_path)

        for i in range(1, 6):  # 5 levels of rain
            augmented_image = add_rain_albumentations(image, intensity=i)

            new_image_name = f"{name}r{i}{ext}"
            new_label_name = f"{name}r{i}.txt"

            cv2.imwrite(os.path.join(output_val_image_folder, new_image_name), augmented_image)

            if os.path.exists(label_path):
                shutil.copy(label_path, os.path.join(output_val_label_folder, new_label_name))

print("✅ Validation rain augmentation completed.")

✅ Validation rain augmentation completed.


In [10]:
# === TRAINING WITH YOLOv11 ===

all_metrics = []

# Loop through model versions for training
for model_version in model_versions:
    print(f"Training model version: {model_version}")
    model = YOLO(model_version)
    model.train(data="data.yaml", epochs=100, save_period=10, seed=seed, device=0)

    # === Add Metrics ===
    results = model.val(data="data.yaml")

    metrics = {
        "model": model_version,
        "precision": results.box.mp,
        "recall": results.box.mr,
        "f1_score": results.box.f1,
        "mAP_0.5": results.box.map50,
        "mAP_0.5_0.95": results.box.map,
        "fitness": results.box.fitness
    }

    all_metrics.append(metrics)

# Save after all are done
pd.DataFrame(all_metrics).to_csv("yolov11_rain_results.csv", index=False)


Training model version: yolo11n.pt
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plot

train: Scanning D:\fyp\Desktop\Drone\drone_dataset\augmented\labels... 5060 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5060/5060 [00:03<00:00, 1610.95it/s]


train: New cache created: D:\fyp\Desktop\Drone\drone_dataset\augmented\labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.00.0 ms, read: 25.815.5 MB/s, size: 154.9 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid_augmented\labels... 1735 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1735/1735 [00:01<00:00, 1366.84it/s]

val: New cache created: D:\fyp\Desktop\Drone\drone_dataset\valid_augmented\labels.cache


...


100 epochs completed in 6.239 hours.
Optimizer stripped from runs\detect\train15\weights\last.pt, 114.4MB
Optimizer stripped from runs\detect\train15\weights\best.pt, 114.4MB

Validating runs\detect\train15\weights\best.pt...
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
YOLO11x summary (fused): 190 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 55/55 [00:09<00:00,  5.64it/s]


                   all       1735       1845      0.924      0.843      0.902      0.538
Speed: 0.1ms preprocess, 3.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\train15
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
YOLO11x summary (fused): 190 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1855.31602.0 MB/s, size: 97.9 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid_augmented\labels.cache... 1735 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1735/1735 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 109/109 [00:15<00:00,  7.19it/s]


                   all       1735       1845      0.925      0.844      0.903      0.539
Speed: 0.1ms preprocess, 7.0ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\train152


In [12]:
# === PREDICTIONS ===
#paths = glob.glob("D:/Softwares_Code/MLProject/DroneDetect/drone_dataset/valid/images/*")
paths = glob.glob("D:/fyp/Desktop/Drone/drone_dataset/valid_augmented/images/*")


In [13]:
n = 8
results = model.predict(paths[:n])
for i in range(n):
    r = results[i]
    img = Image.fromarray(r.plot())
    plt.figure(dpi=100)
    plt.imshow(img)
    plt.axis("off")
    plt.tight_layout()
    plt.show()


0: 640x640 5 drones, 9.1ms
1: 640x640 5 drones, 9.1ms
2: 640x640 5 drones, 9.1ms
3: 640x640 5 drones, 9.1ms
4: 640x640 5 drones, 9.1ms
5: 640x640 1 drone, 9.1ms
6: 640x640 1 drone, 9.1ms
7: 640x640 1 drone, 9.1ms
Speed: 1.4ms preprocess, 9.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [46]:
######################
"""  Sepearate Rain augmented level 1 - 5 so i can get mAP each level. """
# Only the valid images needed, no need retrain.

# === Input paths ===
input_val_image_folder = "D:/fyp/Desktop/Drone/drone_dataset/valid/images"
input_val_label_folder = "D:/fyp/Desktop/Drone/drone_dataset/valid/labels"

# === Rain level loop ===
for i in range(1, 6):  # Rain Level 1 to 5
    output_val_image_folder = f"D:/fyp/Desktop/Drone/drone_dataset/valid_augmented_rain{i}/images"
    output_val_label_folder = f"D:/fyp/Desktop/Drone/drone_dataset/valid_augmented_rain{i}/labels"

    os.makedirs(output_val_image_folder, exist_ok=True)
    os.makedirs(output_val_label_folder, exist_ok=True)

    # === Process each image ===
    for filename in os.listdir(input_val_image_folder):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            name, ext = os.path.splitext(filename)
            image_path = os.path.join(input_val_image_folder, filename)
            label_path = os.path.join(input_val_label_folder, name + ".txt")

            image = cv2.imread(image_path)

            # Apply rain of level i
            augmented_image = add_rain_albumentations(image, intensity=i)

            # Save image & label
            new_image_name = f"{name}{ext}"  # Keep original name
            new_label_name = f"{name}.txt"

            cv2.imwrite(os.path.join(output_val_image_folder, new_image_name), augmented_image)

            if os.path.exists(label_path):
                shutil.copy(label_path, os.path.join(output_val_label_folder, new_label_name))

    print(f"✅ Rain Level {i} augmentation completed.")

✅ Rain Level 1 augmentation completed.
✅ Rain Level 2 augmentation completed.
✅ Rain Level 3 augmentation completed.
✅ Rain Level 4 augmentation completed.
✅ Rain Level 5 augmentation completed.


In [52]:
############### #

# === Trained model checkpoints ===
model_versions = {
    "yolo11n.pt": "D:/fyp/Desktop/Drone/runs/detect/train11/weights/best.pt",
    "yolo11s.pt": "D:/fyp/Desktop/Drone/runs/detect/train12/weights/best.pt",
    "yolo11m.pt": "D:/fyp/Desktop/Drone/runs/detect/train13/weights/best.pt",
    "yolo11l.pt": "D:/fyp/Desktop/Drone/runs/detect/train14/weights/best.pt",
    "yolo11x.pt": "D:/fyp/Desktop/Drone/runs/detect/train15/weights/best.pt",
}

# === Rain Level + No Rain YAMLs ===
rain_levels = {
    "No Rain": "data_norain.yaml",
    "Rain Level 1": "data_rain1.yaml",
    "Rain Level 2": "data_rain2.yaml",
    "Rain Level 3": "data_rain3.yaml",
    "Rain Level 4": "data_rain4.yaml",
    "Rain Level 5": "data_rain5.yaml",
}

# === Results container ===
all_metrics = []

# === Evaluate each model on each rain level ===
for model_name, model_path in model_versions.items():
    model = YOLO(model_path)  # Load trained model

    for level_name, data_file in rain_levels.items():
        print(f"📊 Evaluating {model_name} on {level_name}...")

        results = model.val(data=data_file)

        metrics = {
            "model": model_name,
            "rain_level": level_name,
            "precision": results.box.mp.item(),
            "recall": results.box.mr.item(),
            "f1_score": results.box.f1.item(),
            "mAP_0.5": results.box.map50.item(),
            "mAP_0.5_0.95": results.box.map.item(),
            "fitness": results.box.fitness()
        }

        all_metrics.append(metrics)

# === Save results to CSV ===
df = pd.DataFrame(all_metrics)
df.to_csv("yolov11_rain_all_results.csv", index=False)

print("\n✅ All metrics saved to yolov11_rain_all_results.csv")



📊 Evaluating yolo11n.pt on No Rain...
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 2131.51337.2 MB/s, size: 186.5 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid\labels.cache... 347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 347/347 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:03<00:00,  6.45it/s]


                   all        347        369       0.92      0.899      0.932      0.555
Speed: 0.3ms preprocess, 6.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\val77
📊 Evaluating yolo11n.pt on Rain Level 1...
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 1838.2740.2 MB/s, size: 94.8 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid_augmented_rain1\labels.cache... 347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 347/347 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:03<00:00,  6.52it/s]


                   all        347        369      0.958      0.856      0.931      0.566
Speed: 0.3ms preprocess, 6.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\val78
📊 Evaluating yolo11n.pt on Rain Level 2...
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 899.6396.2 MB/s, size: 97.0 KB)


...

                   all        347        369      0.941      0.835      0.902      0.538
Speed: 0.1ms preprocess, 64.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\val104
📊 Evaluating yolo11x.pt on Rain Level 4...
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 2313.4989.7 MB/s, size: 143.6 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid_augmented_rain4\labels.cache... 347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 347/347 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:23<00:00,  1.06s/it]


                   all        347        369      0.924      0.851      0.906      0.541
Speed: 0.1ms preprocess, 64.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\val105
📊 Evaluating yolo11x.pt on Rain Level 5...
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 2502.11173.5 MB/s, size: 263.3 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid_augmented_rain5\labels.cache... 347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 347/347 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:23<00:00,  1.06s/it]


                   all        347        369      0.888      0.818      0.891      0.538
Speed: 0.2ms preprocess, 64.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\val106

✅ All metrics saved to yolov11_rain_all_results.csv


In [59]:
# Averaging all results.

# Load your CSV file
df = pd.read_csv("yolov11_rain_all_results.csv")

# Filter out the "No Rain" rows
df_rain_only = df[df["rain_level"] != "No Rain"]

# Group by model and compute average for rain levels only
avg_rain_metrics = df_rain_only.groupby("model").mean(numeric_only=True).reset_index()

# Save to a new CSV file
avg_rain_metrics.to_csv("yolov11_rain_only_avg_metrics.csv", index=False)

# Print results
print("🌧️ Averaged Metrics per Model (Rain Levels Only):")
print(avg_rain_metrics)

🌧️ Averaged Metrics per Model (Rain Levels Only):
        model  precision    recall  f1_score   mAP_0.5  mAP_0.5_0.95   fitness
0  yolo11l.pt   0.922616  0.824678  0.870785  0.903825      0.556090  0.590864
1  yolo11m.pt   0.904774  0.865835  0.884638  0.922826      0.556252  0.592909
2  yolo11n.pt   0.932528  0.846218  0.886920  0.918783      0.556015  0.592291
3  yolo11s.pt   0.916400  0.843902  0.878582  0.912934      0.555506  0.591249
4  yolo11x.pt   0.924154  0.842425  0.881356  0.901051      0.540005  0.576110


In [56]:
"""
this is where i put the fine tuning, i select the best version model,
then add (3) fine tuning.

(1) """

# Identify the best YOLOv11 version, based on all the accuracy metrics
# df = pd.read_csv("yolov11_rain_results.csv")
df = pd.read_csv("yolov11_rain_all_results.csv")
best_model = df.sort_values(by="fitness", ascending=False).iloc[0]["model"]
print("Best YOLOv11 model:", best_model)

# === FINE-TUNING THE BEST MODEL ===
print(f"🔧 Fine-tuning best model: {best_model}")

fine_tune_model = YOLO(best_model)

# Optional: increase epochs, reduce learning rate for fine-tuning
fine_tune_model.train(
    data="data.yaml",
    epochs=50,  # smaller range, just to refine
    lr0=0.0004,   # smaller learning rate for fine-tuning
    lrf=0.01,   # cosine LR schedule final ratio
    warmup_epochs=1,
    momentum=0.937,
    weight_decay=0.0005,
    patience=5,   # early stopping if no improvement
    seed=seed,
    device=0,
    save=True,
    save_period=10,
    name="fine_tuned_model"
)

Best YOLOv11 model: yolo11n.pt
🔧 Fine-tuning best model: yolo11n.pt
New https://pypi.org/project/ultralytics/8.3.155 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0004, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=Fal

train: Scanning D:\fyp\Desktop\Drone\drone_dataset\augmented\labels.cache... 5060 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5060/5060 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access  (ping: 0.00.0 ms, read: 2027.4839.0 MB/s, size: 154.9 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid_augmented\labels.cache... 1735 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1735/1735 [00:00<?, ?it/s]


...

      16/50      2.65G       1.04     0.6541      1.285         11        640: 100%|██████████| 317/317 [00:16<00:00, 19.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 55/55 [00:03<00:00, 14.34it/s]


                   all       1735       1845      0.861      0.808      0.861       0.47
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 11, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

16 epochs completed in 0.093 hours.
Optimizer stripped from runs\detect\fine_tuned_model14\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\fine_tuned_model14\weights\best.pt, 5.5MB

Validating runs\detect\fine_tuned_model14\weights\best.pt...
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 55/55 [00:04<00:00, 13.10it/s]


                   all       1735       1845      0.879      0.844      0.894      0.533
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\fine_tuned_model14


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000299B668CA90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [57]:
# Re-Evaluate After Fine-Tuning

#results_finetuned = fine_tune_model.val(data="data.yaml")

rain_levels = {
    "No Rain": "data_norain.yaml",
    "Rain Level 1": "data_rain1.yaml",
    "Rain Level 2": "data_rain2.yaml",
    "Rain Level 3": "data_rain3.yaml",
    "Rain Level 4": "data_rain4.yaml",
    "Rain Level 5": "data_rain5.yaml",
}

# === Store all evaluation metrics ===
results_finetuned = []

# === Loop over each rain level ===
for level_name, yaml_file in rain_levels.items():
    print(f"📊 Validating fine-tuned model on {level_name}...")
    results = fine_tune_model.val(data=yaml_file)

    metrics = {
        "model": "fine_tuned_best",
        "rain_level": level_name,
        "precision": results.box.mp.item(),
        "recall": results.box.mr.item(),
        "f1_score": results.box.f1.item(),
        "mAP_0.5": results.box.map50.item(),
        "mAP_0.5_0.95": results.box.map.item(),
        "fitness": results.box.fitness()
    }

    results_finetuned.append(metrics)

# === Save all to CSV ===
df_finetuned = pd.DataFrame(results_finetuned)
df_finetuned.to_csv("yolov11_finetuned_rainlevel_results.csv", index=False)

print("\n✅ Fine-tuned results saved to 'yolov11_finetuned_rainlevel_results.csv'")


📊 Validating fine-tuned model on No Rain...
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 2184.21486.9 MB/s, size: 1471.7 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid\labels.cache... 347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 347/347 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:01<00:00, 16.25it/s]


                   all        347        369       0.88      0.851      0.893      0.531
Speed: 0.3ms preprocess, 1.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\fine_tuned_model142
📊 Validating fine-tuned model on Rain Level 1...
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 2014.0632.0 MB/s, size: 149.1 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid_augmented_rain1\labels.cache... 347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 347/347 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:01<00:00, 17.35it/s]


                   all        347        369      0.903      0.863      0.914      0.548
Speed: 0.3ms preprocess, 1.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\fine_tuned_model143
📊 Validating fine-tuned model on Rain Level 2...
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 909.7364.2 MB/s, size: 42.7 KB)


...

                   all        347        369      0.863       0.87      0.901      0.544
Speed: 0.3ms preprocess, 0.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\fine_tuned_model145
📊 Validating fine-tuned model on Rain Level 4...
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 2773.4804.7 MB/s, size: 293.0 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid_augmented_rain4\labels.cache... 347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 347/347 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:01<00:00, 17.58it/s]


                   all        347        369      0.885      0.843      0.894      0.527
Speed: 0.3ms preprocess, 1.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\fine_tuned_model146
📊 Validating fine-tuned model on Rain Level 5...
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 1411.6304.6 MB/s, size: 210.2 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid_augmented_rain5\labels.cache... 347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 347/347 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:01<00:00, 17.70it/s]


                   all        347        369      0.879      0.808      0.868      0.503
Speed: 0.3ms preprocess, 1.0ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\fine_tuned_model147

✅ Fine-tuned results saved to 'yolov11_finetuned_rainlevel_results.csv'


In [ ]:
# Compare Before & After

# Save all results, not just the fine tuned ones.
"""
df.loc[len(df)] = {
    "model": f"{best_model}_finetuned",
    "precision": results_finetuned.box.mp.item(),
    "recall": results_finetuned.box.mr.item(),
    "f1_score": results_finetuned.box.f1.item(),
    "mAP_0.5": results_finetuned.box.map50.item(),
    "mAP_0.5_0.95": results_finetuned.box.map.item(),
    #"fitness": results_finetuned.box.fitness.item()
    "fitness": results_finetuned.box.fitness()  # ← call it like a function
}
df.to_csv("yolov11_finetuned_rain_results.csv", index=False)
"""



In [25]:
# Re-evaluate the model to get results object
results = model.val(data="data.yaml")  # Or replace with fine_tune_model.val(...)

# Extract class names and per-class mAP@0.5:0.95
class_mAPs = []
print("\n📊 Per-Class mAP@0.5:0.95:\n")
print(f"{'Class':<20} {'mAP@0.5:0.95'}")
print("-" * 35)

for i, class_name in results.names.items():
    mAP_class = results.box.maps[i].item()  # Convert from numpy.float32
    class_mAPs.append((class_name, mAP_class))
    print(f"{class_name:<20} {mAP_class:.3f}")

Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 2629.11493.9 MB/s, size: 138.3 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid_augmented\labels.cache... 1735 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1735/1735 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 109/109 [00:15<00:00,  7.17it/s]


                   all       1735       1845      0.925      0.844      0.903      0.539
Speed: 0.1ms preprocess, 7.0ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\train153

📊 Per-Class mAP@0.5:0.95:

Class                mAP@0.5:0.95
-----------------------------------
drone                0.539


In [27]:
# Auto compare.

# Define your model names
model_names = ["yolo11n.pt", "yolo11s.pt", "yolo11m.pt", "yolo11l.pt", "yolo11x.pt"]

# Initialize dataframe
columns = ["model", "precision", "recall", "f1_score", "mAP_0.5", "mAP_0.5_0.95", "fitness"]
df = pd.DataFrame(columns=columns)

# Evaluation function
def evaluate_model(model_path, label):
    model = YOLO(model_path)
    results = model.val(data="data.yaml", imgsz=640, save=False, verbose=False)

    return {
        "model": label,
        "precision": results.box.mp.item(),
        "recall": results.box.mr.item(),
        "f1_score": results.box.f1.item(),
        "mAP_0.5": results.box.map50.item(),
        "mAP_0.5_0.95": results.box.map.item(),
        "fitness": results.box.fitness()
    }

# Loop through base and fine-tuned models
for model_name in model_names:
    base_metrics = evaluate_model(model_name, model_name)
    fine_tuned_path = model_name.replace(".pt", ".pt_finetuned")

    try:
        fine_tuned_metrics = evaluate_model(fine_tuned_path, f"{model_name}_finetuned")
    except FileNotFoundError:
        print(f"⚠️ Fine-tuned model not found: {fine_tuned_path}")
        fine_tuned_metrics = None

    df.loc[len(df)] = base_metrics
    if fine_tuned_metrics:
        df.loc[len(df)] = fine_tuned_metrics

# Save to CSV
df.to_csv("model_comparison_results.csv", index=False)
print("\n✅ Results saved to model_comparison_results.csv")

Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 2962.8678.0 MB/s, size: 167.9 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid_augmented\labels.cache... 1735 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1735/1735 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 109/109 [00:04<00:00, 23.95it/s]


                   all       1735       1845     0.0139      0.313     0.0117    0.00484
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\val5
⚠️ Fine-tuned model not found: yolo11n.pt_finetuned
Ultralytics 8.3.154  Python-3.11.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
YOLO11s summary (fused): 100 layers, 9,443,760 parameters, 0 gradients, 21.5 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 3253.4603.9 MB/s, size: 169.1 KB)


val: Scanning D:\fyp\Desktop\Drone\drone_dataset\valid_augmented\labels.cache... 1735 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1735/1735 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   4%|▎         | 4/109 [00:00<00:13,  7.88it/s]


KeyboardInterrupt: 